In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import getFamaFrenchFactors as gff
# from statsmodels.regression.rolling import RollingOLS
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

# Step 1: Data Preparation

In [2]:
data = pd.read_csv("25_ME_BM.csv").set_index('date')
data.index = pd.to_datetime(data.index, format='%Y%m') + pd.offsets.MonthEnd(0)
data = data.apply(lambda x: x/100)
temp = data.T.stack().reset_index()
data = temp.rename(columns={'level_0': 'portfolio', 0: 'ret'})
data

,portfolio,date,ret
0,ME1BM1,1926-07-31,0.046348
1,ME1BM1,1926-08-31,-0.006390
2,ME1BM1,1926-09-30,-0.094350
3,ME1BM1,1926-10-31,-0.075499
4,ME1BM1,1926-11-30,-0.010690
...,...,...,...
28120,ME5BM5,2019-11-30,0.026716
28121,ME5BM5,2019-12-31,0.023242
28122,ME5BM5,2020-01-31,-0.041012
28123,ME5BM5,2020-02-29,-0.056014


**Get FF5 data**

In [3]:
ff5 = gff.famaFrench5Factor(frequency='m')
ff5 = ff5.rename(columns={'date_ff_factors': 'date'})
ff5 = ff5.set_index('date')
ff5

,Mkt-RF,SMB,HML,RMW,CMA,RF
date,,,,,,
1963-07-31,-0.0039,-0.0047,-0.0083,0.0066,-0.0115,0.0027
1963-08-31,0.0507,-0.0079,0.0167,0.0039,-0.0040,0.0025
1963-09-30,-0.0157,-0.0048,0.0018,-0.0076,0.0024,0.0027
1963-10-31,0.0253,-0.0129,-0.0010,0.0275,-0.0224,0.0029
1963-11-30,-0.0085,-0.0084,0.0171,-0.0045,0.0222,0.0027
...,...,...,...,...,...,...
2019-11-30,0.0387,0.0050,-0.0186,-0.0150,-0.0129,0.0012
2019-12-31,0.0277,0.0096,0.0183,0.0021,0.0131,0.0014
2020-01-31,-0.0011,-0.0440,-0.0627,-0.0123,-0.0234,0.0013


In [4]:
df = pd.merge(data, ff5, on=['date'], how='left')
df = df.dropna()
df['excess_ret'] = df['ret'] - df['RF']
df['constant'] = 1
df

,portfolio,date,ret,Mkt-RF,SMB,HML,RMW,CMA,RF,excess_ret,constant
444,ME1BM1,1963-07-31,0.005557,-0.0039,-0.0047,-0.0083,0.0066,-0.0115,0.0027,0.002857,1
445,ME1BM1,1963-08-31,0.028099,0.0507,-0.0079,0.0167,0.0039,-0.0040,0.0025,0.025599,1
446,ME1BM1,1963-09-30,-0.008536,-0.0157,-0.0048,0.0018,-0.0076,0.0024,0.0027,-0.011236,1
447,ME1BM1,1963-10-31,0.026977,0.0253,-0.0129,-0.0010,0.0275,-0.0224,0.0029,0.024077,1
448,ME1BM1,1963-11-30,-0.035450,-0.0085,-0.0084,0.0171,-0.0045,0.0222,0.0027,-0.038150,1
...,...,...,...,...,...,...,...,...,...,...,...
28120,ME5BM5,2019-11-30,0.026716,0.0387,0.0050,-0.0186,-0.0150,-0.0129,0.0012,0.025516,1
28121,ME5BM5,2019-12-31,0.023242,0.0277,0.0096,0.0183,0.0021,0.0131,0.0014,0.021842,1
28122,ME5BM5,2020-01-31,-0.041012,-0.0011,-0.0440,-0.0627,-0.0123,-0.0234,0.0013,-0.042312,1
28123,ME5BM5,2020-02-29,-0.056014,-0.0813,-0.0007,-0.0401,-0.0164,-0.0253,0.0012,-0.057214,1


# Step 2: Rolling regression to get loadings for FF5 factors

In [5]:
def RollOls_params(indices, result, ycol, xcols):
    roll_df = df.loc[indices] # get relevant data frame subset
    result[indices[-1]] = sm.OLS(roll_df[ycol], roll_df[xcols]).fit().params[:]
    return 0 

# define kwargs to be fed to the ols_predict
kwargs = {"xcols": ['Mkt-RF','SMB','HML','RMW','CMA','constant'], 
          "ycol": 'excess_ret', 
          "result": {}}

# iterate id's sub data frames and call ols for rolling windows
df["index"] = df.index
for idx, sub_df in df.groupby("portfolio"):
    sub_df["index"].rolling(60).apply(RollOls_params, kwargs=kwargs)

**Preparing DataFrame by removing first 60 rows in each portfolio to copy regression results over**

In [6]:
df_pred = df.groupby('portfolio', group_keys=False).apply(lambda x:x.iloc[59:])
df_pred

,portfolio,date,ret,Mkt-RF,SMB,HML,RMW,CMA,RF,excess_ret,constant,index
503,ME1BM1,1968-06-30,-0.005511,0.0069,-0.0026,0.0067,-0.0133,0.0268,0.0043,-0.009811,1,503
504,ME1BM1,1968-07-31,-0.046269,-0.0272,-0.0133,0.0548,-0.0300,0.0361,0.0048,-0.051069,1,504
505,ME1BM1,1968-08-31,0.038011,0.0134,0.0229,0.0100,-0.0060,0.0040,0.0042,0.033811,1,505
506,ME1BM1,1968-09-30,0.065105,0.0403,0.0277,0.0024,-0.0203,0.0092,0.0043,0.060805,1,506
507,ME1BM1,1968-10-31,-0.015684,0.0042,-0.0043,0.0289,-0.0130,0.0281,0.0044,-0.020084,1,507
...,...,...,...,...,...,...,...,...,...,...,...,...
28120,ME5BM5,2019-11-30,0.026716,0.0387,0.0050,-0.0186,-0.0150,-0.0129,0.0012,0.025516,1,28120
28121,ME5BM5,2019-12-31,0.023242,0.0277,0.0096,0.0183,0.0021,0.0131,0.0014,0.021842,1,28121
28122,ME5BM5,2020-01-31,-0.041012,-0.0011,-0.0440,-0.0627,-0.0123,-0.0234,0.0013,-0.042312,1,28122
28123,ME5BM5,2020-02-29,-0.056014,-0.0813,-0.0007,-0.0401,-0.0164,-0.0253,0.0012,-0.057214,1,28123


In [7]:
d = kwargs["result"]
# write results(predicted values) back to original df
df_pred["mkt_param"] = [d[key][0] for key,value in kwargs["result"].items()]
df_pred["smb_param"] = [d[key][1] for key,value in kwargs["result"].items()]
df_pred["hml_param"] = [d[key][2] for key,value in kwargs["result"].items()]
df_pred["rmw_param"] = [d[key][3] for key,value in kwargs["result"].items()]
df_pred["cma_param"] = [d[key][4] for key,value in kwargs["result"].items()]
df_pred["const_param"] = [d[key][-1] for key,value in kwargs["result"].items()]
# showing the last 5 computed values
df_pred.head()  # loadings

,portfolio,date,ret,Mkt-RF,SMB,HML,RMW,CMA,RF,excess_ret,constant,index,mkt_param,smb_param,hml_param,rmw_param,cma_param,const_param
503,ME1BM1,1968-06-30,-0.005511,0.0069,-0.0026,0.0067,-0.0133,0.0268,0.0043,-0.009811,1,503,0.916904,1.698936,-0.329794,-0.270389,-0.035619,-0.000119
504,ME1BM1,1968-07-31,-0.046269,-0.0272,-0.0133,0.0548,-0.0300,0.0361,0.0048,-0.051069,1,504,0.913342,1.700258,-0.300928,-0.266959,-0.035687,-0.000316
505,ME1BM1,1968-08-31,0.038011,0.0134,0.0229,0.0100,-0.0060,0.0040,0.0042,0.033811,1,505,0.912495,1.695801,-0.297765,-0.257899,-0.037665,-0.000512
506,ME1BM1,1968-09-30,0.065105,0.0403,0.0277,0.0024,-0.0203,0.0092,0.0043,0.060805,1,506,0.886574,1.697635,-0.226813,-0.141051,-0.037893,-0.001211
507,ME1BM1,1968-10-31,-0.015684,0.0042,-0.0043,0.0289,-0.0130,0.0281,0.0044,-0.020084,1,507,0.875776,1.732689,-0.310700,-0.254523,-0.019979,-0.001951


# Step 3: Cross-sectional regression

In [8]:
def ols_params(data,xcols,ycol):
    return sm.OLS(data[ycol], data[xcols]).fit().params[:]

ycol = 'excess_ret'
xcols= ['mkt_param','smb_param','hml_param','rmw_param','cma_param','constant']
res = df_pred.groupby('date').apply(ols_params, ycol = ycol , xcols=xcols)
res  # risk premium

,mkt_param,smb_param,hml_param,rmw_param,cma_param,constant
date,,,,,,
1968-06-30,-0.053143,-0.018906,0.009024,-0.042164,0.042452,0.073578
1968-07-31,-0.043909,0.001146,0.054077,-0.020537,0.047813,0.001237
1968-08-31,0.019090,0.023371,0.021878,-0.009650,-0.006586,-0.009669
1968-09-30,0.020665,0.017540,0.006255,-0.005665,-0.013292,0.027549
1968-10-31,-0.003526,-0.006805,0.017999,-0.007216,0.012310,0.012977
...,...,...,...,...,...,...
2019-11-30,0.034044,-0.002719,-0.031390,-0.017335,-0.032044,0.008193
2019-12-31,-0.021409,0.029937,0.020257,-0.018725,0.056514,0.041272
2020-01-31,-0.091409,-0.031053,-0.064149,-0.021916,0.002977,0.091799


In [9]:
res_final = pd.merge(df_pred[['portfolio','date']], res, on=['date'], how='left')
# res_final = res_left[xcols_2]
res_final

,portfolio,date,mkt_param,smb_param,hml_param,rmw_param,cma_param,constant
0,ME1BM1,1968-06-30,-0.053143,-0.018906,0.009024,-0.042164,0.042452,0.073578
1,ME1BM1,1968-07-31,-0.043909,0.001146,0.054077,-0.020537,0.047813,0.001237
2,ME1BM1,1968-08-31,0.019090,0.023371,0.021878,-0.009650,-0.006586,-0.009669
3,ME1BM1,1968-09-30,0.020665,0.017540,0.006255,-0.005665,-0.013292,0.027549
4,ME1BM1,1968-10-31,-0.003526,-0.006805,0.017999,-0.007216,0.012310,0.012977
...,...,...,...,...,...,...,...,...
15545,ME5BM5,2019-11-30,0.034044,-0.002719,-0.031390,-0.017335,-0.032044,0.008193
15546,ME5BM5,2019-12-31,-0.021409,0.029937,0.020257,-0.018725,0.056514,0.041272
15547,ME5BM5,2020-01-31,-0.091409,-0.031053,-0.064149,-0.021916,0.002977,0.091799
15548,ME5BM5,2020-02-29,-0.130220,-0.001224,-0.037639,-0.034156,-0.037097,0.053368


**Get loadings from previous time-series regression**

In [10]:
X = df_pred[['portfolio','date','mkt_param','smb_param','hml_param','rmw_param','cma_param','constant']]
X

,portfolio,date,mkt_param,smb_param,hml_param,rmw_param,cma_param,constant
503,ME1BM1,1968-06-30,0.916904,1.698936,-0.329794,-0.270389,-0.035619,1
504,ME1BM1,1968-07-31,0.913342,1.700258,-0.300928,-0.266959,-0.035687,1
505,ME1BM1,1968-08-31,0.912495,1.695801,-0.297765,-0.257899,-0.037665,1
506,ME1BM1,1968-09-30,0.886574,1.697635,-0.226813,-0.141051,-0.037893,1
507,ME1BM1,1968-10-31,0.875776,1.732689,-0.310700,-0.254523,-0.019979,1
...,...,...,...,...,...,...,...,...
28120,ME5BM5,2019-11-30,1.123488,0.034044,0.738564,-0.549038,-0.306402,1
28121,ME5BM5,2019-12-31,1.118743,0.031481,0.736239,-0.546615,-0.323238,1
28122,ME5BM5,2020-01-31,1.139162,0.015544,0.755238,-0.591723,-0.293079,1
28123,ME5BM5,2020-02-29,1.076760,0.043755,0.752627,-0.659608,-0.405780,1


# Step 4: Predictions and Analysis

**Compute fitted average returns**

In [11]:
multiply = pd.DataFrame(X[xcols].values*res_final[xcols].values,
                        columns=xcols, index=X.index)
multiply['y_head'] = multiply.sum(axis=1)
multiply

,mkt_param,smb_param,hml_param,rmw_param,cma_param,constant,y_head
503,-0.048727,-0.032119,-0.002976,0.011401,-0.001512,0.073578,-0.000356
504,-0.040104,0.001949,-0.016273,0.005483,-0.001706,0.001237,-0.049416
505,0.017420,0.039632,-0.006515,0.002489,0.000248,-0.009669,0.043605
506,0.018321,0.029777,-0.001419,0.000799,0.000504,0.027549,0.075530
507,-0.003088,-0.011790,-0.005592,0.001837,-0.000246,0.012977,-0.005903
...,...,...,...,...,...,...,...
28120,0.038248,-0.000093,-0.023183,0.009517,0.009818,0.008193,0.042501
28121,-0.023951,0.000942,0.014914,0.010235,-0.018267,0.041272,0.025145
28122,-0.104130,-0.000483,-0.048448,0.012968,-0.000872,0.091799,-0.049165
28123,-0.140215,-0.000054,-0.028328,0.022530,0.015053,0.053368,-0.077646


**Combine fitted values with portfolio and dates for further analysis**

In [12]:
prediction = pd.concat([df_pred[['portfolio','date','excess_ret']], multiply['y_head']], axis=1).reset_index(drop=True)

prediction['y_head_lag'] = prediction.groupby('portfolio')['y_head'].shift(1)
prediction = prediction.dropna()
prediction = prediction.drop(columns='y_head')

prediction['correct'] = prediction['excess_ret']*prediction['y_head_lag']
prediction['abs_ret'] = abs(prediction.excess_ret)
prediction

,portfolio,date,excess_ret,y_head_lag,correct,abs_ret
1,ME1BM1,1968-07-31,-0.051069,-0.000356,0.000018,0.051069
2,ME1BM1,1968-08-31,0.033811,-0.049416,-0.001671,0.033811
3,ME1BM1,1968-09-30,0.060805,0.043605,0.002651,0.060805
4,ME1BM1,1968-10-31,-0.020084,0.075530,-0.001517,0.020084
5,ME1BM1,1968-11-30,0.060057,-0.005903,-0.000355,0.060057
...,...,...,...,...,...,...
15545,ME5BM5,2019-11-30,0.025516,0.004740,0.000121,0.025516
15546,ME5BM5,2019-12-31,0.021842,0.042501,0.000928,0.021842
15547,ME5BM5,2020-01-31,-0.042312,0.025145,-0.001064,0.042312
15548,ME5BM5,2020-02-29,-0.057214,-0.049165,0.002813,0.057214


### Average monthly returns (%) of portfolios based on predictions and T-statistics for the risk premiums

- **Jul 1968 to Dec 1991**

In [13]:
# First available date = 1968-07-31
# Last available date = 2020-03-31

start_date = pd.to_datetime('1968-07-31')
end_date = pd.to_datetime('1991-12-31')

gain = prediction[(prediction['date'] >= start_date) & (prediction['date'] <= end_date)][prediction['correct']>0].groupby('portfolio')['abs_ret']
loss = prediction[(prediction['date'] >= start_date) & (prediction['date'] <= end_date)][prediction['correct']<0].groupby('portfolio')['abs_ret']
test = (gain.sum() - loss.sum()) / (gain.count() + loss.count())
test = test.rename('ave_ret')*100

print('Average monthly returns (%) between', start_date.date(), 'and', end_date.date())
# test.to_excel('1968.07 to 1991.12.xlsx')
print(test)

temp = res[(res.index >= start_date) & (res.index <= end_date)]
tstat = abs(temp.mean() / (temp.std()/np.sqrt(len(temp))))
tstat = pd.DataFrame(tstat.rename('t-stats'))

tstat['p-Value'] = tstat['t-stats'].apply(lambda x: stats.t.sf(np.abs(x), len(temp)-1)*2)
tstat = tstat.rename(index=dict(zip(tstat.index.values,['Mkt-RF','SMB','HML','RMW','CMA','constant'])))
print('\nT-statistics of risk premiums between', start_date.date(), 'and', end_date.date())
# tstat.to_excel('tstat 1968.07 to 1991.12.xlsx')
round(tstat, 5)

Average monthly returns (%) between 1968-07-31 and 1991-12-31
portfolio
ME1BM1    2.308117
ME1BM2    1.969990
ME1BM3    1.689452
ME1BM4    1.336631
ME1BM5    1.608397
ME2BM1    1.624859
ME2BM2    1.407588
ME2BM3    1.346405
ME2BM4    0.917986
ME2BM5    1.145379
ME3BM1    1.244042
ME3BM2    1.231327
ME3BM3    0.892974
ME3BM4    0.889841
ME3BM5    0.899738
ME4BM1    0.748444
ME4BM2    0.898855
ME4BM3    0.638655
ME4BM4    0.587810
ME4BM5    0.683207
ME5BM1    0.465290
ME5BM2    0.449647
ME5BM3    0.327440
ME5BM4    0.201848
ME5BM5    0.369703
Name: ave_ret, dtype: float64

T-statistics of risk premiums between 1968-07-31 and 1991-12-31


,t-stats,p-Value
Mkt-RF,2.00709,0.04570
SMB,0.26244,0.79318
HML,2.58243,0.01032
RMW,0.90610,0.36566
CMA,1.14576,0.25287
constant,4.46943,0.00001


- **Jan 1991 to Dec 2013**

In [14]:
# First available date = 1968-07-31
# Last available date = 2020-03-31

start_date = pd.to_datetime('1991-01-01')
end_date = pd.to_datetime('2013-12-31')

gain = prediction[(prediction['date'] >= start_date) & (prediction['date'] <= end_date)][prediction['correct']>0].groupby('portfolio')['abs_ret']
loss = prediction[(prediction['date'] >= start_date) & (prediction['date'] <= end_date)][prediction['correct']<0].groupby('portfolio')['abs_ret']
test = (gain.sum() - loss.sum()) / (gain.count() + loss.count())
test = test.rename('ave_ret')*100

print('Average monthly returns (%) between', start_date.date(), 'and', end_date.date())
# test.to_excel('1991.01 to 2013.12.xlsx')
print(test)

temp = res[(res.index >= start_date) & (res.index <= end_date)]
tstat = abs(temp.mean() / (temp.std()/np.sqrt(len(temp))))
tstat = pd.DataFrame(tstat.rename('t-stats'))

tstat['p-Value'] = tstat['t-stats'].apply(lambda x: stats.t.sf(np.abs(x), len(temp)-1)*2)
tstat = tstat.rename(index=dict(zip(tstat.index.values,['Mkt-RF','SMB','HML','RMW','CMA','constant'])))
print('\nT-statistics of risk premiums between', start_date.date(), 'and', end_date.date())
# tstat.to_excel('tstat 1991.01 to 2013.12.xlsx')
round(tstat, 5)

Average monthly returns (%) between 1991-01-01 and 2013-12-31
portfolio
ME1BM1    1.938095
ME1BM2    1.757608
ME1BM3    1.511377
ME1BM4    1.733829
ME1BM5    2.233063
ME2BM1    0.882497
ME2BM2    1.008427
ME2BM3    0.914643
ME2BM4    1.062873
ME2BM5    1.298679
ME3BM1    0.471404
ME3BM2    0.766701
ME3BM3    0.788673
ME3BM4    0.841458
ME3BM5    0.865911
ME4BM1    0.914458
ME4BM2    0.673079
ME4BM3    0.939195
ME4BM4    0.602346
ME4BM5    0.932974
ME5BM1    0.412133
ME5BM2    0.405055
ME5BM3    0.444600
ME5BM4    0.481744
ME5BM5    0.776804
Name: ave_ret, dtype: float64

T-statistics of risk premiums between 1991-01-01 and 2013-12-31


,t-stats,p-Value
Mkt-RF,2.82426,0.00509
SMB,1.36511,0.17333
HML,1.07912,0.28148
RMW,1.89538,0.05909
CMA,1.51796,0.13017
constant,6.60071,0.00000


- **Jul 1968 to Dec 2013**

In [15]:
# First available date = 1968-07-31
# Last available date = 2020-03-31

start_date = pd.to_datetime('1968-07-01')
end_date = pd.to_datetime('2013-12-31')

gain = prediction[(prediction['date'] >= start_date) & (prediction['date'] <= end_date)][prediction['correct']>0].groupby('portfolio')['abs_ret']
loss = prediction[(prediction['date'] >= start_date) & (prediction['date'] <= end_date)][prediction['correct']<0].groupby('portfolio')['abs_ret']
test = (gain.sum() - loss.sum()) / (gain.count() + loss.count())
test = test.rename('ave_ret')*100

print('Average monthly returns (%) between', start_date.date(), 'and', end_date.date())
# test.to_excel('1968.07 to 2013.12.xlsx')
print(test)

temp = res[(res.index >= start_date) & (res.index <= end_date)]
tstat = abs(temp.mean() / (temp.std()/np.sqrt(len(temp))))
tstat = pd.DataFrame(tstat.rename('t-stats'))

tstat['p-Value'] = tstat['t-stats'].apply(lambda x: stats.t.sf(np.abs(x), len(temp)-1)*2)
tstat = tstat.rename(index=dict(zip(tstat.index.values,['Mkt-RF','SMB','HML','RMW','CMA','constant'])))
print('\nT-statistics of risk premiums between', start_date.date(), 'and', end_date.date())
# tstat.to_excel('tstat 1968.07 to 2013.12.xlsx')
round(tstat, 5)

Average monthly returns (%) between 1968-07-01 and 2013-12-31
portfolio
ME1BM1    2.119095
ME1BM2    1.862016
ME1BM3    1.604159
ME1BM4    1.550368
ME1BM5    1.937739
ME2BM1    1.270126
ME2BM2    1.216162
ME2BM3    1.121528
ME2BM4    0.987367
ME2BM5    1.227225
ME3BM1    0.881126
ME3BM2    1.012196
ME3BM3    0.866021
ME3BM4    0.864632
ME3BM5    0.924760
ME4BM1    0.873054
ME4BM2    0.811336
ME4BM3    0.808686
ME4BM4    0.604024
ME4BM5    0.837640
ME5BM1    0.471423
ME5BM2    0.473146
ME5BM3    0.411959
ME5BM4    0.356912
ME5BM5    0.607232
Name: ave_ret, dtype: float64

T-statistics of risk premiums between 1968-07-01 and 2013-12-31


,t-stats,p-Value
Mkt-RF,3.17709,0.00157
SMB,1.01031,0.31279
HML,2.62327,0.00895
RMW,2.32572,0.02040
CMA,2.11077,0.03525
constant,7.36708,0.00000


- **Jan 2013 to Dec 2019**

In [16]:
# First available date = 1968-07-31
# Last available date = 2020-03-31

start_date = pd.to_datetime('2013-01-01')
end_date = pd.to_datetime('2019-12-31')

gain = prediction[(prediction['date'] >= start_date) & (prediction['date'] <= end_date)][prediction['correct']>0].groupby('portfolio')['abs_ret']
loss = prediction[(prediction['date'] >= start_date) & (prediction['date'] <= end_date)][prediction['correct']<0].groupby('portfolio')['abs_ret']
test = (gain.sum() - loss.sum()) / (gain.count() + loss.count())
test = test.rename('ave_ret')*100

print('Average monthly returns (%) between', start_date.date(), 'and', end_date.date())
# test.to_excel('2013.01 to 2019.12.xlsx')
print(test)

temp = res[(res.index >= start_date) & (res.index <= end_date)]
tstat = abs(temp.mean() / (temp.std()/np.sqrt(len(temp))))
tstat = pd.DataFrame(tstat.rename('t-stats'))

tstat['p-Value'] = tstat['t-stats'].apply(lambda x: stats.t.sf(np.abs(x), len(temp)-1)*2)
tstat = tstat.rename(index=dict(zip(tstat.index.values,['Mkt-RF','SMB','HML','RMW','CMA','constant'])))
print('\nT-statistics of risk premiums between', start_date.date(), 'and', end_date.date())
# tstat.to_excel('tstat 2013.01 to 2019.12.xlsx')
round(tstat, 5)

Average monthly returns (%) between 2013-01-01 and 2019-12-31
portfolio
ME1BM1   -0.436731
ME1BM2   -0.516221
ME1BM3   -0.266342
ME1BM4   -0.499894
ME1BM5   -0.040493
ME2BM1    0.017218
ME2BM2   -0.586657
ME2BM3   -0.849980
ME2BM4   -0.978183
ME2BM5   -0.889908
ME3BM1   -0.470115
ME3BM2   -0.509949
ME3BM3   -0.934068
ME3BM4   -0.808633
ME3BM5   -0.716577
ME4BM1   -0.360348
ME4BM2   -0.384936
ME4BM3   -0.477877
ME4BM4   -0.769181
ME4BM5   -0.941836
ME5BM1   -0.064301
ME5BM2   -0.068179
ME5BM3   -0.162993
ME5BM4   -0.023362
ME5BM5   -0.423426
Name: ave_ret, dtype: float64

T-statistics of risk premiums between 2013-01-01 and 2019-12-31


,t-stats,p-Value
Mkt-RF,0.37104,0.71156
SMB,0.51906,0.60510
HML,1.14201,0.25673
RMW,2.35683,0.02079
CMA,1.49830,0.13785
constant,2.39029,0.01910


- **Jul 1968 to Dec 2019**

In [17]:
# First available date = 1968-07-31
# Last available date = 2020-03-31

start_date = pd.to_datetime('1968-07-01')
end_date = pd.to_datetime('2019-12-31')

gain = prediction[(prediction['date'] >= start_date) & (prediction['date'] <= end_date)][prediction['correct']>0].groupby('portfolio')['abs_ret']
loss = prediction[(prediction['date'] >= start_date) & (prediction['date'] <= end_date)][prediction['correct']<0].groupby('portfolio')['abs_ret']
test = (gain.sum() - loss.sum()) / (gain.count() + loss.count())
test = test.rename('ave_ret')*100

print('Average monthly returns (%) between', start_date.date(), 'and', end_date.date())
# test.to_excel('1968.07 to 2019.12.xlsx')
print(test)

temp = res[(res.index >= start_date) & (res.index <= end_date)]
tstat = abs(temp.mean() / (temp.std()/np.sqrt(len(temp))))
tstat = pd.DataFrame(tstat.rename('t-stats'))

tstat['p-Value'] = tstat['t-stats'].apply(lambda x: stats.t.sf(np.abs(x), len(temp)-1)*2)
tstat = tstat.rename(index=dict(zip(tstat.index.values,['Mkt-RF','SMB','HML','RMW','CMA','constant'])))
print('\nT-statistics of risk premiums between', start_date.date(), 'and', end_date.date())
# tstat.to_excel('tstat 1968.07 to 2019.12.xlsx')
round(tstat, 5)

Average monthly returns (%) between 1968-07-01 and 2019-12-31
portfolio
ME1BM1    1.829159
ME1BM2    1.597764
ME1BM3    1.362886
ME1BM4    1.270340
ME1BM5    1.685345
ME2BM1    1.121888
ME2BM2    1.003123
ME2BM3    0.858889
ME2BM4    0.739518
ME2BM5    0.946461
ME3BM1    0.720681
ME3BM2    0.821897
ME3BM3    0.637163
ME3BM4    0.657267
ME3BM5    0.714377
ME4BM1    0.722125
ME4BM2    0.644944
ME4BM3    0.638872
ME4BM4    0.415293
ME4BM5    0.581231
ME5BM1    0.393069
ME5BM2    0.390369
ME5BM3    0.316341
ME5BM4    0.285865
ME5BM5    0.446656
Name: ave_ret, dtype: float64

T-statistics of risk premiums between 1968-07-01 and 2019-12-31


,t-stats,p-Value
Mkt-RF,2.96657,0.00313
SMB,0.76494,0.44460
HML,2.10429,0.03576
RMW,2.95532,0.00324
CMA,1.61680,0.10643
constant,7.47982,0.00000


In [18]:
prediction['profit'] = np.where(prediction.correct < 0, -prediction.abs_ret, prediction.abs_ret)
test = prediction.groupby('date').profit.mean()*100
# test.to_excel('Results-FF5 (Size-BM) Monthly Average Returns.xlsx')
test

date
1968-07-31    -0.787708
1968-08-31    -2.484012
1968-09-30     5.940452
1968-10-31     0.835408
1968-11-30     4.435108
                ...    
2019-11-30     2.453620
2019-12-31     4.109700
2020-01-31    -3.066440
2020-02-29     6.922756
2020-03-31    23.154904
Name: profit, Length: 621, dtype: float64

In [19]:
start_date = pd.to_datetime('1994-01-01')
end_date = pd.to_datetime('2018-12-31')

gain = prediction[(prediction['date'] >= start_date) & (prediction['date'] <= end_date)][prediction['correct']>0].groupby('portfolio')['abs_ret']
loss = prediction[(prediction['date'] >= start_date) & (prediction['date'] <= end_date)][prediction['correct']<0].groupby('portfolio')['abs_ret']
test = (gain.sum() - loss.sum()) / (gain.count() + loss.count())
test = test.rename('ave_ret')*100

print('Average monthly returns (%) between', start_date.date(), 'and', end_date.date())
# test.to_excel('1968.07 to 2019.12.xlsx')
print(test.mean())

temp = res[(res.index >= start_date) & (res.index <= end_date)]
tstat = abs(temp.mean() / (temp.std()/np.sqrt(len(temp))))
tstat = pd.DataFrame(tstat.rename('t-stats'))

tstat['p-Value'] = tstat['t-stats'].apply(lambda x: stats.t.sf(np.abs(x), len(temp)-1)*2)
tstat = tstat.rename(index=dict(zip(tstat.index.values,['Mkt-RF','SMB','HML','RMW','CMA','constant'])))
print('\nT-statistics of risk premiums between', start_date.date(), 'and', end_date.date())
# tstat.to_excel('tstat 1968.07 to 2019.12.xlsx')
round(tstat, 5)

Average monthly returns (%) between 1994-01-01 and 2018-12-31
0.7669184933333334

T-statistics of risk premiums between 1994-01-01 and 2018-12-31


,t-stats,p-Value
Mkt-RF,2.40481,0.01679
SMB,0.67701,0.49892
HML,0.20188,0.84015
RMW,2.68773,0.00760
CMA,1.19420,0.23334
constant,5.91894,0.00000
